# Práctica 8. Visión por computador II

En esta práctica vamos a crear un clasificador capaz de reconocer dígitos manuscritos, para ello utilizaremos el dataset [MNIST](http://yann.lecun.com/exdb/mnist/) que ya hemos utilizado en prácticas anteriores. La parte obligatoria de la práctica se evalúa sobre 6 puntos; además, esta práctica contiene tres ejercicios adicionales incluidos al final de este fichero, y un ejercicio adicional en el notebook adicional. 

Recordad los 5 pasos de un clasificador de imágenes:
1. Preparar el dataset.
2. Separar en conjuntos de entrenamiento, test y validación.
3. Extraer descriptores.
4. Entrenar el modelo.
5. Evaluar el modelo.

En muchas ocasiones los pasos 2 y 3 son intercambiables, y es lo que haremos en esta práctica.

## 0. Carga del dataset y descripción de las imágenes
 
Para cargar el dataset y describir las imágenes vamos a utilizar las clases `CargaYDescribe` (definida en el fichero CargaYDescribe.py de la carpeta p10) y la clase `Descriptor` junto con sus hijas (definidas en el fichero Descriptor.py también dentro de la carpeta p10). Deberás descargar esta carpeta y descomprimirla utilizando el siguiente comando. 



In [50]:
!wget www.unirioja.es/cu/joheras/ia/p10/p10.zip -O p08.zip
!unzip p08.zip

URL transformed to HTTPS due to an HSTS policy
--2021-04-29 21:41:19--  https://www.unirioja.es/cu/joheras/ia/p10/p10.zip
Resolving www.unirioja.es (www.unirioja.es)... 193.144.2.30
Connecting to www.unirioja.es (www.unirioja.es)|193.144.2.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1274 (1.2K) [application/zip]
Saving to: ‘p08.zip’

p08.zip             100%[===================>]   1.24K  --.-KB/s    in 0s      

2021-04-29 21:41:21 (42.9 MB/s) - ‘p08.zip’ saved [1274/1274]

Archive:  p08.zip
replace p10/CargaYDescribe.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: p10/CargaYDescribe.py   
  inflating: p10/Descriptor.py       
  inflating: p10/__init__.py         


Si abres el fichero Descriptor.py (puedes abrir dicho fichero pulsando en la pestaña con el símbolo ">" que aparece a la izquierda de la pantalla y luego pulsando en la pestaña Files), verás la definición de la clase `Descriptor` y de dos clases hijas. La principal característica de la clase `Descriptor` es que tiene un método llamado `describe` que sirve para describir una imagen. Este método es abstracto y lo proporcionan las clases hijas.

##### Pregunta

¿Qué hacen los métodos ``describe`` de las clases RawImage y Projection?

**Respuesta.** En primer lugar, en la clase RawImage lo que hace el método es convertir la imagen en una escala de grises y aplanarla. En la clase Projection se convierte la imagen en una escala de grises, igual que en la clase anterior, pero en este caso no se aplana, sino que se realiza la suma total de sus píxeles a lo ancho y a lo largo de la imagen.

Si ahora abres el fichero CargaYDescribe.py verás la definición de la clase `CargaYDescribe`. Esta clase permite cargar y describir las imágenes de un directorio que está organizado mediante carpetas donde cada carpeta contiene
las imágenes de una clase.

El constructor de esta clase recibe dos parámetros, un objeto de la clase `Descriptor`, lo que nos permite cambiar fácilmente el método que usamos para describir las imágenes (esta es una técnica llamada inyección de dependencias que ya hemos utilizado alguna vez), y un path que es donde se encuentran las carpetas con las imágenes de cada clase.

Definimos una variable donde almacenamos el path donde se encuentran nuestra imágenes. De nuevo, es necesario descargar la carpeta.

In [51]:
!wget www.unirioja.es/cu/joheras/ia/p10/digitos.zip
!unzip digitos.zip

URL transformed to HTTPS due to an HSTS policy
--2021-04-29 21:41:29--  https://www.unirioja.es/cu/joheras/ia/p10/digitos.zip
Resolving www.unirioja.es (www.unirioja.es)... 193.144.2.30
Connecting to www.unirioja.es (www.unirioja.es)|193.144.2.30|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2176634 (2.1M) [application/zip]
Saving to: ‘digitos.zip.1’

digitos.zip.1       100%[===================>]   2.08M  1.74MB/s    in 1.2s    

2021-04-29 21:41:31 (1.74 MB/s) - ‘digitos.zip.1’ saved [2176634/2176634]

Archive:  digitos.zip
replace digitos/0/0.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: digitos/0/0.jpg         
  inflating: digitos/0/1.jpg         
  inflating: digitos/0/10.jpg        
  inflating: digitos/0/100.jpg       
  inflating: digitos/0/101.jpg       
  inflating: digitos/0/102.jpg       
  inflating: digitos/0/103.jpg       
  inflating: digitos/0/104.jpg       
  inflating: digitos/0/105.jpg       
  inflating: digitos/0/106.jpg   

In [52]:
dataset = "digitos/"

Para este ejemplo vamos a utilizar como descriptor el RawImage. 

In [53]:
from p10.Descriptor import *
from p10.CargaYDescribe import *

descriptor = RawImage()
cargaDatos = CargaYDescribe(dataset,descriptor)
(datos,etiquetas) = cargaDatos.cargaYdescribe()

##### Ejercicio

Comprueba que puedes cambiar de descriptor y que todo sigue funcionando correctamente.


In [54]:
descriptor = Projection()

## 1. Separando el conjunto de entrenamiento y el de test

##### Ejercicio

Utiliza la función train_test_split, vista en las prácticas de aprendizaje automático, para partir el dataset en la siguiente proporción: 75% entrenamiento, 25% test. Almacena la partición en las variables:
`trainData`, `testData`, `trainLabels`, `testLabels`. Utiliza como random_state el valor 84 para así obtener siempre los mismos resultados.

In [55]:
from sklearn.model_selection import train_test_split
(trainData, testData, trainLabels, testLabels) = train_test_split(datos,etiquetas,test_size=0.25, random_state=84)

## 2. Entrenando el modelo

Una vez que hemos partido el banco de imágenes, podemos entrenar distintos modelos como vimos en prácticas anteriores. 

**Ejercicio** 

Utiliza KNN y aplica validación cruzada para fijar los hiperparámetros.

In [56]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier


from sklearn.model_selection import cross_val_score

for k in range(1,26,2):
    modelKNN = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(modelKNN,trainData,trainLabels,cv=10)
    print("k=%d, Precisión: %0.2f (+/- %0.2f)" % (k, scores.mean(), scores.std() * 2))

k=1, Precisión: 0.90 (+/- 0.05)
k=3, Precisión: 0.90 (+/- 0.04)
k=5, Precisión: 0.90 (+/- 0.04)
k=7, Precisión: 0.89 (+/- 0.03)
k=9, Precisión: 0.88 (+/- 0.04)
k=11, Precisión: 0.88 (+/- 0.03)
k=13, Precisión: 0.88 (+/- 0.03)
k=15, Precisión: 0.87 (+/- 0.03)
k=17, Precisión: 0.87 (+/- 0.04)
k=19, Precisión: 0.86 (+/- 0.03)
k=21, Precisión: 0.86 (+/- 0.04)
k=23, Precisión: 0.86 (+/- 0.04)
k=25, Precisión: 0.86 (+/- 0.04)


Al probar por cross validation, sale que el numero de n_neighbors óptimo es 5.

**Ejercicio** 

Una vez hayas encontrado los mejores hiperparámetros almacena y entrena el modelo.

In [57]:
modelKNN = KNeighborsClassifier(n_neighbors=3)
modelKNN.fit(trainData, trainLabels)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='uniform')

##### Ejercicio

Utiliza otro de los algoritmos de clasificación vistos en las prácticas anteriores para crear un nuevo modelo.

In [58]:
modelSVM = SVC()
tuned_parameters = [{'kernel': ['sigmoid'], 'gamma': [2, 3],
                     'C': [0.1,1, 10]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
scores = ['precision', 'recall']

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

def randomsearch(model,parameters):
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = RandomizedSearchCV(model, parameters,cv=5)
    clf.fit(trainData, trainLabels)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()

randomsearch(modelSVM, {'kernel': ['sigmoid','linear'], 
                        'gamma': sp_randint(2, 20), 'C': sp_randint(0.1, 100)})

# Tuning hyper-parameters for 0.8625

Best parameters set found on development set:

{'C': 30, 'gamma': 6, 'kernel': 'linear'}



Como se ve, los mejores hiperparámetros que ha encontrado son: {'C': 16, 'gamma': 13, 'kernel': 'linear'}

In [59]:
modelSVM = SVC(C=16, gamma= 13, kernel= 'linear')
modelSVM.fit(trainData, trainLabels)

SVC(C=16, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=13, kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

## 3. Evaluando el modelo

##### Ejercicio
Una vez tienes tu modelo construido evalualo utilizando la función `classification_report` de la librería sklearn.

In [60]:
from sklearn.metrics import classification_report

print("EVALUACIÓN EN CONJUNTO DE TEST USANDO KNN")
modelKNN = modelKNN.predict(testData)
print(classification_report(testLabels, modelKNN))

EVALUACIÓN EN CONJUNTO DE TEST USANDO KNN
              precision    recall  f1-score   support

           0       0.94      1.00      0.97        66
           1       0.90      1.00      0.95        70
           2       0.94      0.94      0.94        71
           3       0.85      0.88      0.86        58
           4       0.94      0.86      0.90        74
           5       0.86      0.86      0.86        58
           6       0.91      1.00      0.95        42
           7       0.95      0.94      0.95        66
           8       0.98      0.77      0.86        69
           9       0.89      0.94      0.91        66

    accuracy                           0.92       640
   macro avg       0.92      0.92      0.92       640
weighted avg       0.92      0.92      0.92       640



##### Ejercicio 

A continuación muestra la matriz de confusión para ver dónde se producen los fallos. 

In [61]:
print("EVALUACIÓN EN CONJUNTO DE TEST USANDO SVC")
modelSVM = modelSVM.predict(testData)
print(classification_report(testLabels, modelSVM))

EVALUACIÓN EN CONJUNTO DE TEST USANDO SVC
              precision    recall  f1-score   support

           0       0.95      0.95      0.95        66
           1       0.92      0.99      0.95        70
           2       0.85      0.94      0.89        71
           3       0.91      0.83      0.86        58
           4       0.88      0.89      0.89        74
           5       0.90      0.90      0.90        58
           6       0.95      0.95      0.95        42
           7       0.95      0.88      0.91        66
           8       0.87      0.86      0.86        69
           9       0.90      0.86      0.88        66

    accuracy                           0.90       640
   macro avg       0.91      0.90      0.91       640
weighted avg       0.91      0.90      0.90       640



##### Ejercicio

Con el modelo KNN somos capaces de conseguir una precisión del 92%, aunque no está mal, se puede mejorar. Intenta construir un modelo que mejore estos resultados, puedes probar también a cambiar de descriptor.

## 4. Guardando y cargando el modelo

El proceso de entrenar un modelo puede ser muy costoso, por lo que conviene guardar el modelo para poder recuperarlo posteriormente. Para ello utilizamos la librería `pickle`.

In [62]:
import pickle

La siguiente instrucción guarda el modelo en el fichero modelo.sav.

In [63]:
pickle.dump(modelKNN, open('modelo.sav', 'wb'))

La siguiente instrucción carga el modelo del fichero modelo.sav.

In [64]:
loaded_model = pickle.load(open('modelo.sav', 'rb'))

Ahora podemos usar el modelo igual que antes.

In [65]:
predictions = modelSVM.predict(testData)
print(classification_report(testLabels, predictions))

AttributeError: ignored

Para descargar el fichero desde Colab a nuestro ordenador, hay que ejecutar las siguientes instrucciones (esta funcionalidad solo está disponible usando como navegador Google Chrome).

In [ ]:
from google.colab import files
files.download('modelo.sav') 

##### Ejercicio

Guarda tu mejor modelo en el fichero `mimodelo.sav`.

###### Ejercicio

Realizar un estudio estadístico utilizando el paquete StatisticalAnalisis visto en la práctica 6. 


##### Ejercicio

En la práctica 5 se proporcionó un fichero que explicaba cómo utilizar la librería `keras` para construir un modelo capaz de predecir los dígitos manuscritos. Utiliza dicha funcionalidad para construir un modelo de predicción. 

##### Ejercicio

El último ejercicio adicional que se propone consiste en combinar la funcionalidad de contornos vista en la práctica 7 y el clasificador de identificación de dígitos. 

Comenzamos cargando las librerías y funcionalidades necesarias. 

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import imutils
%matplotlib inline 

In [ ]:
def mostrarImagen(image):
    if len(image.shape)==3:
        img2 = image[:,:,::-1]
        plt.imshow(img2)
        plt.show()
    else:
        img2 = image
        plt.imshow(img2,cmap='gray')
        plt.show()

Cargamos la imagen en la cual queremos leer los dígitos escritos y la mostramos.

In [ ]:
!wget www.unirioja.es/cu/joheras/ia/p10/imagen1.png
image = cv2.imread("imagen1.png")
mostrarImagen(image)

Convierte la imagen a escala de grises.

Aplica una umbralización con valor de umbral 80.

Busca los cortornos de la imagen, y para cada uno de ellos encuentra el bounding box que lo contiene. 

Utilizando el bounding box que contiene al contorno, predice el número contenido utilizando el modelo que hayas definido y muestra el resultado en la imagen. Debes tener en cuenta varias cosas, las imágenes del dataset tienen tamaño 28x28, por lo que es posible que tengas que reescalar tu imagen, además en las imágenes el digito aparece en blanco con fondo negro, mientras que en la imagen está justo al revés, por lo que tendrás que invertir los colores. 

Guarda este fichero con tus soluciones a los distintos ejercicios usando la opción *"Save in Github..."*. 